In [1]:
import polars as pl

import polars.selectors as cs

In [2]:
df_movies_path = r"F:\Datasets\CSV datasets\movies\movies.csv"
df_movies_reviews_path = r"F:\Datasets\CSV datasets\movies\movies_reviews.csv"

In [3]:
df_movies = pl.read_csv(df_movies_path)
df_reviews = pl.read_csv(df_movies_reviews_path)

In [14]:
df_joined = df_movies.join(
    df_reviews,
    on="id",
    how="inner",
    suffix="_suffix",
)

In [26]:
drop_columns = [column for column in df_joined.columns if column.endswith("_suffix")]

df_joined = df_joined.select(cs.exclude(drop_columns))

In [33]:
df_joined

id,title,releaseDate,rating,genres,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,production_companies,director,writer,top_cast,quote,score,date,author,publicationName,review_type
i64,str,str,str,str,str,i64,str,f64,i64,str,f64,f64,str,str,str,str,str,str,i64,str,str,str,str
2000545497,"""Dekalog (1988)""","""1996-03-22""","""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",572,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""What a bitter pill to swallow …",100,"""2023-11-22""","""Siza""",null,"""user"""
2000545497,"""Dekalog (1988)""","""1996-03-22""","""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",572,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""The best 10 hours you will spe…",100,"""2021-05-01""","""ItsMahdiyarr""",null,"""user"""
2000545497,"""Dekalog (1988)""","""1996-03-22""","""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",572,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""Touching and human, Dekalog is…",100,"""2019-12-05""","""Andremax""",null,"""user"""
2000545497,"""Dekalog (1988)""","""1996-03-22""","""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",572,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""Krzysztof Kieslowski's ""The De…",100,"""2018-04-26""","""thelol""",null,"""user"""
2000545497,"""Dekalog (1988)""","""1996-03-22""","""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",572,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""For me it is simply breathtaki…",100,"""2024-10-31""","""Daki1105""",null,"""user"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2000538890,"""Saint Misbehavin': The Wavy Gr…","""2010-12-10""","""Not Rated""","""Documentary""","""Beginning with Woodstock ‘99, …",87,null,null,7,null,0.0,1.0,null,"""Argot Pictures""","""Michelle Esrick""",null,"""Larry Brilliant,Wavy Gravy,Enz…","""This Age-of-Aquarius relic's d…",40,"""2010-12-10""","""David Fear""","""Time Out""","""critic"""
2000538599,"""Boys to Men""","""2001-04-27""",null,"""Drama""","""Compilation of 4 short films f…",75,null,null,8,null,0.0,1.0,null,"""Jour de Fête Films""",null,null,"""Ema Tuennerman,Brett Chukerman…","""1.Você não fala sobre o Clube …",100,"""2024-09-19""","""Grandecu""",null,"""user"""
2000538594,"""Boogie Woogie""","""2010-04-23""","""R""","""Comedy,Drama""","""A comedy of manners set agains…",94,"""Seduction. Betrayal. Death. Th…",null,11,null,46.0,10.0,"""Mixed or average""","""The Works International,Consta…","""Duncan Ward""","""Danny Moynihan""","""Gillian Anderson,Alan Cumming,…","""A stinging satire of the conte…",50,"""2010-10-25""","""gazpacho""",null,"""user"""
